# Demo: How to use machine to machine (non interactive mode) authentication

The purpose of this notebook is to demonstrate the use of machine-to-machine authentication (OAuth2 client credential flow) with the Serenity SDK.

The expectation is that users will store their credentials securely and then provide them to the SDK to facilitate the authentication process.

In [1]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

The API keys and secrets are generated through Serenity's web UI.

Please follow the instructions provided [in the developer website](https://developer.cloudwall.network/userguide/api-access)

In [2]:
def load_secure_config():
    # The code here returns dummy values as demonstration.
    # Users are expected to perform their credential retrieval securely.
    return {
        "domain":"<domain>",
        "userAudience":"<aud>",
        "url":"https://serenity-rest.$client_name.cloudwall.network",
        "environment":"TEST",
        "clientId":"<client_id>",
        "clientSecret":"<client_secret>"}

Once the credentials are generated and stored securely in your environment. Please follow the next steps to create `SerenityClient` to facilitate the authentication process

In [3]:
from serenity_sdk.client import SerenityClient
from serenity_sdk.client.config import ConnectionConfig, Environment

secure_config = load_secure_config()
config = ConnectionConfig(
    domain=secure_config["domain"],
    user_audience=secure_config['userAudience'],
    client_id=secure_config["clientId"],
    client_secret=secure_config["clientSecret"],
    url=secure_config["url"],
    env=Environment(secure_config['environment'])
)

client = SerenityClient(config)

ConnectionError: HTTPSConnectionPool(host='%3cdomain%3e', port=443): Max retries exceeded with url: /oauth/token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f7eb8180f10>: Failed to resolve '%3cdomain%3e' ([Errno -2] Name or service not known)"))

Next we'll use `SerenityClient` with `SerenityApiProvider` which is provides convenience methods to call Serenity's REST APIs

In [ ]:
import datetime
import pandas as pd
import seaborn as sns
from serenity_sdk.api.provider import SerenityApiProvider

api = SerenityApiProvider(client)

The following code demonstrates how we can use Serenity's SDK API library load factor model outputs

In [ ]:
model_short_name = 'risk.factor.ff.beta.mt.v1_6'
model_meta = api.model().load_model_metadata(datetime.date.today())
model_config_id = model_meta.get_model_configuration_id(model_short_name)

Please note that the latest factor model output are available as **T-1**. The outputs are generated daily and available after **2AM UTC**.

In [ ]:
from serenity_sdk.types.common import CalculationContext

# The as_of_date is the date for the model output which is generally available as T-1.
as_of_date = datetime.date.today() - datetime.timedelta(days=1)

ctx = CalculationContext(as_of_date=as_of_date, model_config_id=model_config_id)

Get the factor returns

In [ ]:
api.risk().get_factor_returns(ctx)

Get the factor correlations

In [ ]:
factor_corr_df = api.risk().get_factor_correlation_matrix(ctx)
display(factor_corr_df)

Get the factor covariances

In [ ]:
factor_cov_df = api.risk().get_factor_covariance_matrix(ctx)
display(factor_cov_df)

Let's use asset master to provide human readable names when displaying the assets for the Asset Factor Exposure

In [ ]:
asset_master = api.refdata().load_asset_master()

Finally, get the factor loadings (exposures) for the entire asset universe

In [ ]:
exposures_df = api.risk().get_asset_factor_exposures(ctx, asset_master, None)
display(exposures_df)

# END